<center><font size=4> Mask Detection from  Image

In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils import paths
import numpy as np
import cv2
import os
from overLay.overLap import drawEmoji

Load model & test data

In [2]:
CONF = .5
imagePaths = list(paths.list_images("images"))
prototxtPath = r'faceDetector/deploy.prototxt'
weightsPath = r'faceDetector/res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

model = load_model(r'models/maskDetector11.model') # our trained model in root directory

 pre-process image & Check our detections result Funtion

In [3]:

def detect(image):
#     image = cv2.imread(imagePaths[4]) # we are just testing 1st image 
    
    withMaskColor, withoutMaskColor  = (0, 255, 0), (0, 0, 255)
    font = cv2.FONT_HERSHEY_SIMPLEX

    cpyImg = image.copy()
    (h, w, _)= image.shape

    #blob for image (image, scale, resize, location of face)
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0 ))
    faceNet.setInput(blob)
    detections = faceNet.forward()


    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > CONF:
            #bounding box
            bbox = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = bbox.astype("int")

            #ensure bounding box within dimensions of frame
            (startX, startY)= (max(0, startX), max(0, startY))
            (endX, endY) = (min(w-1, endX), min(h-1, endY))

            #process to feed into model
            face = image[startY: endY, startX: endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (244,244))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            (mask, withoutMash) = model.predict(face)[0]

            # display result

            label = "Mask" if mask> withoutMash else "No Mask"
            color = withMaskColor if label=="Mask" else withoutMaskColor

            #adding accouricy
            label = "{}: {:.2f}%".format(label, max(mask, withoutMash) * 100)

            #display label along with rectangle
            cv2.putText(image, label, (startX, startY-10), font, .5, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
            
            # emoji Draw based on mask
            if mask> withoutMash:
                mode = True
            else:
                mode = False
            drawEmoji(image, mode)
    
    return image

In [4]:
cam = cv2.VideoCapture(0)

while True:
    s, frame = cam.read()
    
    if not s:
        break

    frame = detect(frame)
    cv2.imshow("Mask Detectot", frame)
    key = cv2.waitKey(10) & 0xFF
    if key == ord("q"):
        break
        
cam.release()        
cv2.destroyAllWindows()

## we cant detect forground Image becasue 
It’s too obscured by the mask
The dataset used to train the face detector did not contain example images of people wearing face masks
## check on reltime Mask Detection 